In [1]:
using Singular
#using Revise
import Nemo
#using BlockArrays
using LinearAlgebra: I

Singular.jl, based on
                     SINGULAR                                 /  
 A Computer Algebra System for Polynomial Computations       /  Singular.jl: 0.3.1 
                                                           0<   Singular   : 4.2.1-4
 by: W. Decker, G.-M. Greuel, G. Pfister, H. Schoenemann     \   
FB Mathematik der Universitaet, D-67653 Kaiserslautern        \
     


In [4]:
import Catlab.Theories: dom, codom, id, compose, ⋅, ∘, dagger, dunit, dcounit,
  otimes, ⊗, munit, braid, oplus, ⊕, mzero, swap,
  mcopy, Δ, mmerge, ∇, delete, ◊, create, □, plus, zero, coplus, cozero,
  pair, copair, proj1, proj2, coproj1, coproj2, meet, join, top, bottom

Ok, so it appears that I can get modules over complex number polynomials from singular.
I can get the syzygies, which corresponds ~ to a nullspace calculation




In [5]:
CC = Nemo.ComplexField(64)
P, (s,) = PolynomialRing(CC, ["s"])
i = Nemo.onei(CC) # P(i) ? The imaginary number

#helpers to deal with Singular.jl
eye(m) = P.(Matrix{Int64}(I, m, m)) # There is almost certainly a better way of doing this. Actually dispatching Matrix?
zayro(m,n) = P.(zeros(Int64,m,n)) #new zeros method?
mat1(m::Int64) = fill(P(m), (1,1) )
mat1(m::Float64) = fill(P(m), (1,1) )
mat1(m::spoly{Singular.n_unknown{Nemo.acb}}) = fill(m, (1,1))

# Objects are the dimensionality of the vector space
struct DynOb
    m::Int
end

# Linear relations represented 
struct DynMorph
  input::Array{spoly{Singular.n_unknown{Nemo.acb}},2}
  output::Array{spoly{Singular.n_unknown{Nemo.acb}},2}
end

dom(x::DynMorph) = DynOb(size(x.input)[2])
codom(x::DynMorph) = DynOb(size(x.output)[2])
id(X::DynOb) = DynMorph(eye(X.m), -eye(X.m))

# https://discourse.julialang.org/t/syntax-for-constructing-a-1x1-matrix/5207/11

#Base.transpose(x::DynMorph) = DynMorph(x.output, x.input)





#addboy = DynMorph( [mat1(1) mat1(1)] , mat1(-1) ) # add together the inputs
plus(X::DynOb) = DynMorph( [eye(X.m) eye(X.m)] , - eye(X.m) )


mcopy(X::DynOb) = Dyn( [eye(X.m) ; eye(X.m)] , -eye(2*X.m) ) # copy input
#copyboy = DynMorph( [mat1(1) ; mat1(1)] , -eye(2) ) # copy input
delete(A::DynOb) = DynMorph( fill(P.(0),(0,A.m)) , fill(P.(0),(0,0)) )   
create(A::DynOb) = DynMorph( fill(P.(0),(0,0)) , fill(P.(0),(0,A.m)) )
dagger(x::DynMorph) = DynMorph(x.output, x.input)

dunit(A::DynOb) = compose(create(A), mcopy(A))
dcounit(A::DynOb) = compose(mmerge(A), delete(A))


scale(M) = DynMorph( mat1(M),mat1(-1))
diff =  scale(i*s) #DynMorph(mat1(i*s) , mat1(-1)) # could also lift to m dim like idd # scale(i*s)
integ = dagger(diff)
#cupboy = DynMorph( [mat1(1) mat1(-1)] , fill(P.(0),(1,0)) )
#capboy = transpose(cupboy)

#terminal

# relational operations
# The meet
# Inclusion

# I think this is a nullspace calculation?
# almost all the context is trying to work around Singular's interface to one i can understand
function quasinullspace(A)
   rows, cols = size(A)
   vs = Array(gens(Singular.FreeModule(P, rows)))
   q = [sum(A[:,i] .* vs) for i in 1:cols]
   M = Singular.Module(P,q...)
   S = Singular.Matrix(syz(M)) # syz is the only meat of the computation
   return Base.transpose([S[i,j] for j=1:Singular.ncols(S), i=1:Singular.nrows(S) ])
end

function compose(x::DynMorph,y::DynMorph) 
    nx, xi = size(x.input)
    nx1, xo = size(x.output)
    @assert nx1 == nx
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    A = [ x.input                x.output P.(zeros(Int64,nx,yo)) ;
          P.(zeros(Int64,ny,xi)) y.input  y.output    ]
    B = quasinullspace(A)
    projB = [B[1:xi       ,:] ;
             B[xi+yi+1:end,:] ]
    C = Base.transpose(quasinullspace(Base.transpose(projB)))
    return DynMorph( C[:, 1:xi] ,C[:,xi+1:end] )
end

# basically the direct sum. The monoidal product of linear relations
function otimes( x::DynMorph, y::DynMorph) 
    nx, xi = size(x.input)
    nx1, xo = size(x.output)
    @assert nx1 == nx
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    return DynMorph( [ x.input                P.(zeros(Int64,nx,yi));
                       P.(zeros(Int64,ny,xi)) y.input               ],
                      [x.output                P.(zeros(Int64,nx,yo));
                       P.(zeros(Int64,ny,xo))  y.output               ])
    
end

# munit(::Type{FinOrdRelOb}) = FinOrdRelOb(1)

otimes (generic function with 32 methods)

In [6]:
function mydiag(X)
    compose(mcopy(X),id(X) ⊗ id(X))
end

mydiag (generic function with 1 method)

In [9]:
using Catlab.Theories
X = Ob(FreeAbelianBicategoryRelations, :X)
mydiag(X)

MethodError: MethodError: no method matching ⊗(::Catlab.Theories.FreeAbelianBicategoryRelations.Hom{:id}, ::Catlab.Theories.FreeAbelianBicategoryRelations.Hom{:id})

In [36]:
compose(par(idd(1),idd(1)), cupboy)

DynMorph(spoly{Singular.n_unknown{Nemo.acb}}[(-1.0000000000000000000 + i*0) (1.0000000000000000000 + i*0)], Array{spoly{Singular.n_unknown{Nemo.acb}}}(undef,1,0))

In [ ]:
cap

In [33]:
cupboy

DynMorph(spoly{Singular.n_unknown{Nemo.acb}}[(1.0000000000000000000 + i*0) (-1.0000000000000000000 + i*0)], Array{spoly{Singular.n_unknown{Nemo.acb}}}(undef,0,0))

In [ ]:
using Compose
using Catlab.WiringDiagrams, Catlab.Graphics
#using Catlab
import Convex, SCS
using Catlab.Theories

A, B, C, D = Ob(FreeSymmetricMonoidalCategory, :A, :B, :C, :D)
f, g = Hom(:f, A, B), Hom(:g, B, A);
diff = Hom(:∂, A, A)
integ = Hom(:∫ , A, A)

to_composejl(diff ⋅ integ, background_color="lightgray", props=Dict(
  :box => [fill("white"), stroke("black")],
))

In [ ]:

#to_composejl(plus(X) ⋅ mcopy(X))

In [8]:
mydiag(X)

MethodError: MethodError: no method matching ⊗(::Catlab.Theories.FreeAbelianBicategoryRelations.Hom{:id}, ::Catlab.Theories.FreeAbelianBicategoryRelations.Hom{:id})

In [9]:
methods(otimes)

# 31 methods for generic function "otimes":
[1] otimes(A::Catlab.CategoricalAlgebra.FinRelations.FinOrdRelOb, B::Catlab.CategoricalAlgebra.FinRelations.FinOrdRelOb) in Catlab.CategoricalAlgebra.FinRelations at /home/philip/.julia/packages/Catlab/z8n1h/src/categorical_algebra/FinRelations.jl:125
[2] otimes(xs::Array{T,1}) where T in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:35
[3] otimes(A::Catlab.Theories.FreeSymmetricMonoidalCategory.Ob, B::Catlab.Theories.FreeSymmetricMonoidalCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:77
[4] otimes(f::Catlab.Theories.FreeSymmetricMonoidalCategory.Hom, g::Catlab.Theories.FreeSymmetricMonoidalCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:78
[5] otimes(A::Catlab.Theories.FreeCartesianCategory.Ob, B::Catlab.Theories.FreeCartesianCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:137
[6] otimes(f::Catlab.Theories.FreeCartesianCategory.Hom, g::Catlab.Theories.FreeCartesianCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:138
[7] otimes(A::Catlab.Theories.FreeBiproductCategory.Ob, B::Catlab.Theories.FreeBiproductCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:196
[8] otimes(f::Catlab.Theories.FreeBiproductCategory.Hom, g::Catlab.Theories.FreeBiproductCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:197
[9] otimes(A::Catlab.Theories.FreeClosedMonoidalCategory.Ob, B::Catlab.Theories.FreeClosedMonoidalCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:234
[10] otimes(f::Catlab.Theories.FreeClosedMonoidalCategory.Hom, g::Catlab.Theories.FreeClosedMonoidalCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:235
[11] otimes(A::Catlab.Theories.FreeCartesianClosedCategory.Ob, B::Catlab.Theories.FreeCartesianClosedCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:275
[12] otimes(f::Catlab.Theories.FreeCartesianClosedCategory.Hom, g::Catlab.Theories.FreeCartesianClosedCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:276
[13] otimes(A::Catlab.Theories.FreeCompactClosedCategory.Ob, B::Catlab.Theories.FreeCompactClosedCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:312
[14] otimes(f::Catlab.Theories.FreeCompactClosedCategory.Hom, g::Catlab.Theories.FreeCompactClosedCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:313
[15] otimes(A::Catlab.Theories.FreeDaggerSymmetricMonoidalCategory.Ob, B::Catlab.Theories.FreeDaggerSymmetricMonoidalCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:376
[16] otimes(f::Catlab.Theories.FreeDaggerSymmetricMonoidalCategory.Hom, g::Catlab.Theories.FreeDaggerSymmetricMonoidalCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:377
[17] otimes(A::Catlab.Theories.FreeDaggerCompactCategory.Ob, B::Catlab.Theories.FreeDaggerCompactCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:404
[18] otimes(f::Catlab.Theories.FreeDaggerCompactCategory.Hom, g::Catlab.Theories.FreeDaggerCompactCategory.Hom) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:405
[19] otimes(A::Catlab.Theories.FreeTracedMonoidalCategory.Ob, B::Catlab.Theories.FreeTracedMonoidalCategory.Ob) in Catlab.Theories at /home/philip/.julia/packages/Catlab/z8n1h/src/theories/Monoidal.jl:426
[20] otimes(f::Catlab.Theories.FreeTracedMonoidalCategory.Hom, g::Catlab.Theories

In [7]:
methods(mcopy)

# 18 methods for generic function "mcopy":
[1] mcopy(A::Catlab.CategoricalAlgebra.FinRelations.FinOrdRelOb) in Catlab.CategoricalAlgebra.FinRelations at /home/philip/.julia/packages/Catlab/z8n1h/src/categorical_algebra/FinRelations.jl:146
[2] mcopy(x1::Catlab.Theories.FreeCartesianCategory.Ob) in Catlab.Theories
[3] mcopy(x1::Catlab.Theories.FreeBiproductCategory.Ob) in Catlab.Theories
[4] mcopy(x1::Catlab.Theories.FreeCartesianClosedCategory.Ob) in Catlab.Theories
[5] mcopy(x1::Catlab.Theories.FreeBicategoryRelations.Ob) in Catlab.Theories
[6] mcopy(x1::Catlab.Theories.FreeAbelianBicategoryRelations.Ob) in Catlab.Theories
[7] mcopy(m::Catlab.CategoricalAlgebra.Matrices.MatrixDom) in Catlab.CategoricalAlgebra.Matrices at /home/philip/.julia/packages/Catlab/z8n1h/src/categorical_algebra/Matrices.jl:61
[8] mcopy(A::Ports{Any,Value} where Value, n::Int64) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:227
[9] mcopy(A::Ports{MonoidalCategoryWithDiagonals,Value} where Value, n::Int64) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:244
[10] mcopy(A::Ports{CartesianCategory,Value} where Value, n::Int64) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:246
[11] mcopy(A::Ports{MonoidalCategoryWithBidiagonals,Value} where Value, n::Int64) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:260
[12] mcopy(A::Ports{BiproductCategory,Value} where Value, n::Int64) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:263
[13] mcopy(A::Ports{BicategoryRelations,Value} where Value, n::Int64) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:319
[14] mcopy(A::Ports{AbelianBicategoryRelations,Value} where Value, n::Int64) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:343
[15] mcopy(A::Ports) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/z8n1h/src/wiring_diagrams/Algebraic.jl:232
[16] mcopy(x1::Catlab.LinearAlgebra.GraphicalLinearAlgebra.FreeLinearFunctions.Ob) in Catlab.LinearAlgebra.GraphicalLinearAlgebra
[17] mcopy(x1::Catlab.LinearAlgebra.GraphicalLinearAlgebra.FreeLinearRelations.Ob) in Catlab.LinearAlgebra.GraphicalLinearAlgebra
[18] mcopy(x1::Catlab.LinearAlgebra.StructuredGraphicalLinearAlgebra.FreeStructuredLinearFunctions.Ob) in Catlab.LinearAlgebra.StructuredGraphicalLinearAlgebra

In [8]:
compose(integ,diff)

DynMorph(spoly{Singular.n_unknown{Nemo.acb}}[(-1.0000000000000000000 + i*0)], spoly{Singular.n_unknown{Nemo.acb}}[(1.0000000000000000000 + i*0)])

In [13]:
? ⊗

"⊗" can be typed by \otimes<tab>

search:

Couldn't find ⊗
Perhaps you meant P, i, s, !, %, &, *, +, -, /, :, <, >, \, ^, |, ~, ÷, π or ℯ


No documentation found.

Binding `⊗` does not exist.


In [10]:
compose(compose(scale(2), scale(3)), scale(4))

DynMorph(spoly{Singular.n_unknown{Nemo.acb}}[([-24.00000000000000000 +/- 7.38e-18] + i*0)], spoly{Singular.n_unknown{Nemo.acb}}[(1.0000000000000000000 + i*0)])

In [4]:
CC = Nemo.ComplexField(64)
i = Nemo.onei(CC)
P, (s,) = PolynomialRing(CC, ["s"])

(Singular Polynomial Ring (Coeffs(19)),(s),(dp(1),C), spoly{Singular.n_unknown{Nemo.acb}}[s])

In [5]:
M = Singular.FreeModule(P, 2)
input,output = gens(M)
i * s * input - output

[ i*s  -1 ]

[1 -1]
#scacle
[M  1]
#transpose

1×2 Array{Any,2}:
 (0 + i*1.0000000000000000000)*s  -1

In [ ]:
function compose( f,g  ) 
    [f.input f.output  0;
     0       g.input   g.output]


In [ ]:
A = BlockArray(rand(4, 5), [1,3], [2,3])
#A[Block(1,2)]
#blockaxes(A)
#BlockArrays.blocklengths(A)
blocksize(A)
A[Block(1,1)]
blockaxes(A)

In [ ]:
struct DynMorph
    input
    output
    labels
    objs
end

In [18]:
id = DynMorph([1], [-1])
two = DynMorph([2],[-1])



DynMorph([2], [-1])

In [56]:
using LinearAlgebra

In [157]:
function quasinullspace(A)
   rows, cols = size(A)
   vs = Array(gens(Singular.FreeModule(P, rows)))
   q = [sum(A[:,i] .* vs) for i in 1:cols]
   M = Singular.Module(P,q...)
   S = Singular.Matrix(syz(M)) # syz is the only meat of the computation
   return Base.transpose([S[i,j] for j=1:Singular.ncols(S), i=1:Singular.nrows(S) ])
end

A = P.(CC.([1 -1  0 ; 
                           0  1 -1]))
A = P.([s -1  0 ; 
        0  s -1])
A1 = quasinullspace(A)
println(A1)
A * A1



A2 = quasinullspace(Base.transpose(A1))


Base.transpose(A2)
#println(A2)
#Base.transpose(A1) * A2

Aproj = [A1[1:1,:] ; 
         A1[3:end,:]]
quasinullspace(Base.transpose(Aproj)) # looks ok?

spoly{Singular.n_unknown{Nemo.acb}}[(1.0000000000000000000 + i*0); s; s^2]


2×1 Transpose{spoly{Singular.n_unknown{Nemo.acb}},Array{spoly{Singular.n_unknown{Nemo.acb}},2}}:
 s^2
 (-1.0000000000000000000 + i*0)

In [136]:
A = [1 2 ; 3 4]

#A = P.(CC.([A A zeros(2,2) ; zeros(2,2) A A]))
#A = Base.transpose(P.(CC.([1 -1  0 ; 
#                           0  1 -1])))

A = Base.transpose(s .* (CC.([1 -1  0 ; 
                              0  1 -1])))
rows, cols = size(A)
vs = Array(gens(Singular.FreeModule(P, cols)))
#dot(A[2,:], vs)

q = [sum(A[i,:] .* vs) for i in 1:rows]
M = Singular.Module(P,q...)
S = Singular.Matrix(syz(M))
S = [S[i,j] for j=1:Singular.ncols(S), i=1:Singular.nrows(S) ]
projS = Base.transpose([ S[:,1:1] S[:,3:end]])
#projS
vs = Array(gens(Singular.FreeModule(P, 1)))
q = [sum(projS[i,:] .* vs) for i in 1:2]

M = Singular.Module(P,q...)
Singular.Matrix(syz(M))
#projS
#A * vs
#Singular.Module(P, A * vs)



[(-1.0000000000000000000 + i*0)
(1.0000000000000000000 + i*0)]

In [29]:



#Matrix{}
i

UndefVarError: UndefVarError: i not defined

In [82]:
typeof(idd(3).input)

Array{spoly{Singular.n_unknown{Nemo.acb}},2}

In [83]:
Array{spoly{Singular.n_unknown{Nemo.acb}},2}

Array{spoly{Singular.n_unknown{Nemo.acb}},2}

In [79]:
Array(1:10)[3:6]

4-element Array{Int64,1}:
 3
 4
 5
 6

In [74]:
Base.show(io::IO, z::DynMorph) = print(io, " $(z.input)  |  $(z.output)" )

In [75]:
compose(idd(3),idd(3)).output

spoly{Singular.n_unknown{Nemo.acb}}[(1.0000000000000000000 + i*0) 0 0; 0 (1.0000000000000000000 + i*0) 0; 0 0 (1.0000000000000000000 + i*0); (1.0000000000000000000 + i*0) 0 0; 0 (1.0000000000000000000 + i*0) 0; 0 0 (1.0000000000000000000 + i*0); (1.0000000000000000000 + i*0) 0 0; 0 (1.0000000000000000000 + i*0) 0; 0 0 (1.0000000000000000000 + i*0)]

3×3 Array{spoly{Singular.n_unknown{Nemo.acb}},2}:
 (1.0000000000000000000 + i*0)  …  0
 0                                 0
 0                                 (1.0000000000000000000 + i*0)

In [54]:
P.(zeros(Int64,3,2))

3×2 Array{spoly{Singular.n_unknown{Nemo.acb}},2}:
 0  0
 0  0
 0  0

In [41]:
P.(Matrix{Int64}(I, 2, 2))

2×2 Array{spoly{Singular.n_unknown{Nemo.acb}},2}:
 (1.0000000000000000000 + i*0)  0
 0                              (1.0000000000000000000 + i*0)

In [ ]:
id(m) = DynMorph(I, -I)

In [ ]:
using Catlab.GAT
using LinearAlgebra: I

struct DynDomain
  dim::Int
end

@instance Category(MatrixDomain, Matrix) begin
    dom(x::DynMorph) = DynDomain(size(x.input)[2])
    codom(x::DynMorph) = DynDomain(size(x.output)[2])

  id(m::DynDomain) = Matrix{m.eltype}(I, m.dim, m.dim)
  compose(M::Matrix, N::Matrix) = M*N
end

In [ ]:
(x::DynMorph  ⊗  y::DynMorph) = DynMorph( [x.input  0 ;
                                             0      y.input] ,
                                          [x.output 0 ;
                                             0      y.output] )
copy(n) =  DynMorph( [I ; I],  [-I 0 ; 0 -I]  )
#copy dual to intersect.
cup(n) = DynMorph( [ I -I ], []  )
#cap dual to cup





In [ ]:
scale(M) = DynMorph([ M -1 ])
diff = DynMorph([i*s] , [-1])
integ = transpose(diff)



In [8]:
using Compose

∫

In [28]:
?Category

search: Category Category2 CategoryExpr CategoryWithProducts



Theory of *categories* (with no extra structure)

**Warning**: We compose functions from left to right, i.e., if f:A→B and g:B→C then compose(f,g):A→C. Under this convention function are applied on the right, e.g., if a∈A then af∈B.

We retain the usual meaning of the symbol ∘ (\circ), i.e., g∘f = compose(f,g). This usage is too entrenched to overturn, inconvenient though it may be. We use symbol ⋅ (\cdot) for diagrammatic composition: f⋅g = compose(f,g).


In [27]:
using Catlab.GAT
using LinearAlgebra: I

struct MatrixDomain
  eltype::Type
  dim::Int
end

@instance Category(MatrixDomain, Matrix) begin
  dom(M::Matrix) = MatrixDomain(eltype(M), size(M,1))
  codom(M::Matrix) = MatrixDomain(eltype(M), size(M,2))

  id(m::MatrixDomain) = Matrix{m.eltype}(I, m.dim, m.dim)
  compose(M::Matrix, N::Matrix) = M*N
end

ErrorException: error in method definition: function Theories.⋅ must be explicitly imported to be extended

In [22]:
dump(diff ⋅ integ)

Catlab.Theories.FreeSymmetricMonoidalCategory.Hom{:compose}
  args: Array{Catlab.Theories.FreeSymmetricMonoidalCategory.Hom{:generator}}((2,))
    1: Catlab.Theories.FreeSymmetricMonoidalCategory.Hom{:generator}
      args: Array{Any}((3,))
        1: Symbol ∂
        2: Catlab.Theories.FreeSymmetricMonoidalCategory.Ob{:generator}
          args: Array{Symbol}((1,))
            1: Symbol A
          type_args: Array{Catlab.Syntax.GATExpr}((0,))
        3: Catlab.Theories.FreeSymmetricMonoidalCategory.Ob{:generator}
          args: Array{Symbol}((1,))
            1: Symbol A
          type_args: Array{Catlab.Syntax.GATExpr}((0,))
      type_args: Array{Catlab.Syntax.GATExpr}((2,))
        1: Catlab.Theories.FreeSymmetricMonoidalCategory.Ob{:generator}
          args: Array{Symbol}((1,))
            1: Symbol A
          type_args: Array{Catlab.Syntax.GATExpr}((0,))
        2: Catlab.Theories.FreeSymmetricMonoidalCategory.Ob{:generator}
          args: Array{Symbol}((1,))
            1: 

In [20]:
to_composejl(diff ⋅ integ, background_color="lightgray", props=Dict(
  :box => [fill("white"), stroke("black")],
))

In [9]:
to_composejl(f⋅g, background_color="lightgray", props=Dict(
  :box => [fill("white"), stroke("black")],
))

In [3]:
to_composejl(f⋅g)

In [6]:
to_composejl(f⊗g)

In [10]:
Base.transpose(x::DynMorph) = DynMorph(x.output,x.input)

In [33]:


v1 = vector(P, s + 1, s*s + 1, s)
v2 = vector(P, s^2 + 1, 2s + 3s, s)

M = Singular.Module(P, v1, v2)
G = std(M; complete_reduction=true)

mysyz = syz(M)

Singular Module over Singular Polynomial Ring (Coeffs(19)),(s),(dp(1),C), with Generators:
0

In [37]:
Singular.MatrixSpace(P,3,3)

MethodError: MethodError: no method matching Singular.MatrixSpace(::Singular.PolyRing{Singular.n_unknown{acb}}, ::Int64, ::Int64)

In [29]:
vector(P, [s, s, 1 + s]...)

s*gen(3)+s*gen(2)+s*gen(1)+gen(3)

In [38]:
v1 = vector(P, 1, 0, 0, 0)
i1 = vector(P, 0, 1, 0 ,0)
v2 = vector(P, 0, 0, 1, 0)
i2 = vector(P, 0, 0, 0 ,1)




MethodError: MethodError: no method matching vector(::Singular.PolyRing{Singular.n_unknown{acb}}, ::Int64, ::Int64, ::Int64, ::Int64)
Closest candidates are:
  vector(::Singular.PolyRing{T}, !Matched::spoly{T}...) where T<:RingElem at /home/philip/.julia/packages/Singular/GGfzi/src/module/vector.jl:229

In [47]:
M = Singular.FreeModule(P, 4)
v1 = M([P(1), P(0), P(0),P(0)])
i1 = M([P(0), P(1), P(0),P(0)])
v2 = M([P(0), P(0), P(1),P(0)])
i2 = M([P(0), P(0), P(0),P(1)])

e1 = (v1 - v2) + 10 * i1 # current through resistor
e2 = 10i * s * v2 +  (i1 - i2)  # current conservation + capacitor.


(0 + i*10.000000000000000000)*s*gen(3)+(-1.0000000000000000000 + i*0)*gen(4)+gen(2)

In [53]:
v1,i1,v2,i2 = gens(M)
e1 = (v1 - v2) + 10 * i1 # current through resistor
e2 = 10i * s * v2 +  (i1 - i2)  # current conservation + capacitor.
G = Singular.Module(P,e1,e2)
syz(G) # no I need teh syzygies of the transpose of this matrix.

(0 + i*10.000000000000000000)*s*gen(3)+(-1.0000000000000000000 + i*0)*gen(4)+gen(2)

In [88]:
transpose(hcat([Array(G[i]) for i in 1:Singular.ngens(G)]...))

2×4 LinearAlgebra.Transpose{spoly{Singular.n_unknown{acb}},Array{spoly{Singular.n_unknown{acb}},2}}:
 (1.0000000000000000000 + i*0)  …  0
 0                                 (-1.0000000000000000000 + i*0)

In [107]:
G2 = Singular.Matrix(G)
M2 = Singular.Module(P, [  vector(P, [ G2[i,j] for j in 1:Singular.ncols(G2) ]...)  for i in 1:Singular.nrows(G2) ]...)
Singular.Matrix(syz(M2)) # I think these are the V-Rep of the currents and voltages

[(-10.000000000000000000 + i*0), (-10.000000000000000000 + i*0)*s+(0 + i*[0.100000000000000000 +/- 1.22e-20])
(1.0000000000000000000 + i*0), s
0, (0 + i*[0.100000000000000000 +/- 1.22e-20])
(1.0000000000000000000 + i*0), 0]

In [96]:
Singular.ncols())

4

In [87]:
?transpose

search: transpose transpose! n_transExt_to_spoly



```
transpose(A)
```

Lazy transpose. Mutating the returned object should appropriately mutate `A`. Often, but not always, yields `Transpose(A)`, where `Transpose` is a lazy transpose wrapper. Note that this operation is recursive.

This operation is intended for linear algebra usage - for general data manipulation see [`permutedims`](@ref Base.permutedims), which is non-recursive.

# Examples

```jldoctest
julia> A = [3+2im 9+2im; 8+7im  4+6im]
2×2 Array{Complex{Int64},2}:
 3+2im  9+2im
 8+7im  4+6im

julia> transpose(A)
2×2 Transpose{Complex{Int64},Array{Complex{Int64},2}}:
 3+2im  8+7im
 9+2im  4+6im
```

---

```
transpose(x::Mat)
```

> Return the transpose of the given matrix.


---

```
transpose(x::MatAlgElem{T}) where T <: RingElement
```

> Return the transpose of the given matrix.


---

transpose(M::smatrix{T}) where T <: AbstractAlgebra.RingElem

> Given a matrix $M=(m_{ij})_{i, j}$, return the matrix $M^T=(m_{ji})_{j, i}$.



In [79]:
rank(G)

4

In [46]:
P(s) # so i should just map P over everything to be safe

s

In [66]:
?Matrix

search: Matrix matrix MatrixElem MatrixSpace matrix_repr MatrixAlgebra smatrix



```
Matrix{T} <: AbstractMatrix{T}
```

Two-dimensional dense array with elements of type `T`, often used to represent a mathematical matrix. Alias for [`Array{T,2}`](@ref).

---

```
Matrix{T}(undef, m, n)
```

Construct an uninitialized [`Matrix{T}`](@ref) of size `m`×`n`. See [`undef`](@ref).

# Examples

```julia-repl
julia> Matrix{Float64}(undef, 2, 3)
2×3 Array{Float64,2}:
 6.93517e-310  6.93517e-310  6.93517e-310
 6.93517e-310  6.93517e-310  1.29396e-320
```

---

```
Matrix{T}(nothing, m, n)
```

Construct a [`Matrix{T}`](@ref) of size `m`×`n`, initialized with [`nothing`](@ref) entries. Element type `T` must be able to hold these values, i.e. `Nothing <: T`.

# Examples

```jldoctest
julia> Matrix{Union{Nothing, String}}(nothing, 2, 3)
2×3 Array{Union{Nothing, String},2}:
 nothing  nothing  nothing
 nothing  nothing  nothing
```

---

```
Matrix{T}(missing, m, n)
```

Construct a [`Matrix{T}`](@ref) of size `m`×`n`, initialized with [`missing`](@ref) entries. Element type `T` must be able to hold these values, i.e. `Missing <: T`.

# Examples

```jldoctest
julia> Matrix{Union{Missing, String}}(missing, 2, 3)
2×3 Array{Union{Missing, String},2}:
 missing  missing  missing
 missing  missing  missing
```

---

```
Matrix(A::MatrixElem)
```

> Convert `A` to a Julia `Matrix` of the same dimensions with the same elements.


# Examples

```jldoctest; setup = :(using AbstractAlgebra)
julia> A = ZZ[1 2 3; 4 5 6]
[1  2  3]
[4  5  6]

julia> Matrix(A)
2×3 Array{BigInt,2}:
 1  2  3
 4  5  6
```


In [31]:
M
M = FreeModule(R, 3)
v2 = M([x + 1, x*y + 1, y])
V = Array(v1)

Singular Module over Singular Polynomial Ring (Coeffs(19)),(s),(dp(1),C), with Generators:
s^2*gen(2)+s*gen(3)+s*gen(1)+gen(2)+gen(1)
s^2*gen(1)+s*gen(3)+(5.0000000000000000000 + i*0)*s*gen(2)+gen(1)

In [23]:
M[2] # ok I have access to the inidivudal boys

s^2*gen(1)+s*gen(3)+(5.0000000000000000000 + i*0)*s*gen(2)+gen(1)

In [19]:
R, (s,) = PolynomialRing(QQ, ["s"])

v1 = vector(R, s + 1, s*s + 1, s)
v2 = vector(R, s^2 + 1, 2s + 3s, s)

M = Singular.Module(R, v1, v2)

G = std(M; complete_reduction=true)

mysyz = syz(M)

Singular Module over Singular Polynomial Ring (QQ),(s),(dp(1),C), with Generators:
0

In [5]:
struct BIzboy
   ins
   outs
end







i0 * R = v2 - v1

jC  

ok here's an immediate glitch. I don't see ocmplex numbers in singular

maybe I can access it via nemo, but nemo already has smith normnal form?

v1    v2     v3

In [6]:
module Experiment
    struct Foo
            bar :: Int
    end
end

Main.Experiment

In [8]:
Experiment.Foo(3)

Main.Experiment.Foo(3)

In [7]:
R, (x, y) = PolynomialRing(QQ, ["x", "y"])

(Singular Polynomial Ring (QQ),(x,y),(dp(2),C), spoly{n_Q}[x, y])

In [11]:
x^2 + 1

x^2+1

In [14]:
I = Ideal(R, x^2 +1, x*y + 1)

Singular Ideal over Singular Polynomial Ring (QQ),(x,y),(dp(2),C) with generators (x^2+1, x*y+1)

In [22]:
G = std(I)
Z = syz(G)
F = fres(G,0)
F[1]
F[2]


Singular Module over Singular Polynomial Ring (QQ),(x,y),(dp(2),C), with Generators:
-y^2*gen(1)+x*gen(2)-y*gen(2)-gen(1)

In [23]:
S = @PolynomialRing(ZZ, "x", 5, :deglex) # make a 5 variable polynomial

Singular Polynomial Ring (ZZ),(x1,x2,x3,x4,x5),(Dp(5),C)

In [37]:
R, (x, y) = PolynomialRing(QQ, ["x", "y"])

I = Ideal(R, x^2*y + 2y + 1, y^2 + 1)

F = syz(I)

M = Singular.Matrix(I)
N = Singular.Matrix(F)
println(F)
println(M * N)
println(N)

Singular Module over Singular Polynomial Ring (QQ),(x,y),(dp(2),C), with Generators:
x^2*y*gen(2)-y^2*gen(1)+2*y*gen(2)+gen(2)-gen(1)
[0]
[-y^2-1
x^2*y+2*y+1]


In [4]:
R, (s,) = PolynomialRing(QQ, ["s"])

(Singular Polynomial Ring (QQ),(s),(dp(1),C), spoly{n_Q}[s])

In [6]:
x + 1

MethodError: MethodError: no method matching +(::Array{spoly{n_Q},1}, ::Int64)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(!Matched::Complex{Bool}, ::Real) at complex.jl:297
  +(!Matched::Missing, ::Number) at missing.jl:115
  ...

In [9]:
R, (x, y) = PolynomialRing(QQ, ["x", "y"])

v1 = vector(R, x + 1, x*y + 1, y)
v2 = vector(R, x^2 + 1, 2x + 3y, x)

M = Singular.Module(R, v1, v2)
rank(M)

3

In [10]:
M

Singular Module over Singular Polynomial Ring (QQ),(x,y),(dp(2),C), with Generators:
x*y*gen(2)+x*gen(1)+y*gen(3)+gen(2)+gen(1)
x^2*gen(1)+x*gen(3)+2*x*gen(2)+3*y*gen(2)+gen(1)

In [12]:
fieldnames(M)

MethodError: MethodError: no method matching fieldnames(::smodule{spoly{n_Q}})
Closest candidates are:
  fieldnames(!Matched::Core.TypeofBottom) at reflection.jl:175
  fieldnames(!Matched::Type{#s8} where #s8<:Tuple) at reflection.jl:177
  fieldnames(!Matched::DataType) at reflection.jl:172
  ...